In [1]:
import numpy as np
from oe_acute import MNE
from oe_acute import trial_utils as tu
import os
import pickle
import glob

In [2]:
import numpy as np
from scipy.io.wavfile import read
from scipy import signal
from scipy.optimize import minimize
import glob
import os
import pickle
from oe_acute import trial_utils as tu
import tqdm
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import networkx as nx

In [3]:
exp_path = '/mnt/sphere/share/NCM_acute/experiments/B1240/block3/'
sort_path =  '/mnt/sphere/share/NCM_acute/sorting/B1240/block3/'
stim_path = '/mnt/sphere/share/NCM_acute/MET_prediction_expt/'

ratings = [1,2,3]

ratings_str = ''.join([str(x) for x in ratings])

In [4]:
nperseg=128
noverlap=64
nfft=128
#nsegbins=64
bp_low=300
bp_high=12000
bp_order=5
preemph_b=0.97
ref_level_dB=20
min_level_dB=-30

spec_all = []
stereo_all = []
stim_files = glob.glob(os.path.join(stim_path, '*.wav.sine'))

### Load stim_spec, stim_resp

In [5]:
spec_f = '/mnt/cube/srrudrar/stim_preprocess/temporal_model/B1240/stim_spec.pkl'
with open(spec_f, 'rb') as f:
    stim_spec = pickle.load(f)

resp_f = '/mnt/cube/srrudrar/stim_preprocess/temporal_model/B1240/stim_resp.pkl'
with open(resp_f, 'rb') as f:
    stim_resp = pickle.load(f)

In [6]:
type(stim_spec), type(stim_resp)

(list, list)

### Reduce to 32 bins

In [7]:
np.shape(stim_spec[0]), np.shape(stim_resp[0])

((64, 365), (214, 365))

In [8]:
n_rows = 0
n_cols = 1

stim_spec_16 = []
for stim in stim_spec:
    stim_16 = MNE.kozlov_averaging(stim, n_rows=n_cols, n_cols=n_rows)
    stim_spec_16.append(stim_16)

In [9]:
np.shape(stim_spec_16[0]), np.shape(stim_resp[0])

((32, 365), (214, 365))

### Prepare temporal dset

In [10]:
def prepare_temporal_dset_lag(stim_spec, stim_resp, lag, nsegbins, thresh=0.01):
    '''
    Create the dataset for training the autoencoder
    '''
    
    nfreqbins = stim_spec[0].shape[0]
    stim_spec_resp = []
    segs_list = []
    spec1_list = []

    # normalize response
    for i in range(len(stim_spec)):
        stim_resp[i] /= np.amax(stim_resp[i])
        stim_spec_resp.append(np.vstack([stim_spec[i], stim_resp[i]]))
                              
    for j in range(len(stim_spec_resp)):
        segs_list.append([stim_spec_resp[j][:, n:(n+nsegbins)] for n in range(stim_spec_resp[j].shape[-1] - nsegbins - lag)])
        spec1_list.append([stim_spec_resp[j][:, n+nsegbins+lag] for n in range(stim_spec_resp[j].shape[-1] - nsegbins - lag)])

    for k in range(len(segs_list)):
        indx = [index for index,value in enumerate(segs_list[k]) if (np.mean(value[:nfreqbins, :]) > thresh)]
        #print(type(indx))
        segs_list[k] = [s for s in segs_list[k] if (np.mean(s[:nfreqbins, :]) > thresh)]
        spec1_list[k] = [spec1_list[k][i] for i in indx]

        segs_list[k] = np.stack(segs_list[k], axis=0)
        spec1_list[k] = np.stack(spec1_list[k], axis=0)

    segs = np.vstack(segs_list)
    spec1 = np.vstack(spec1_list)
    
    print('Final shape: {}'.format(segs.shape))
    print('Next shape: {}'.format(spec1.shape))
    return segs, spec1

In [11]:
def prepare_temporal_dset_lag1(stim_spec, stim_resp, lag, nsegbins, thresh=0.01):
    '''
    Create the dataset for training the autoencoder
    '''
    
    nfreqbins = stim_spec[0].shape[0]
    stim_spec_resp = []
    segs_list = []
    spec1_list = []

    # normalize response
    for i in range(len(stim_spec)):
        stim_resp[i] /= np.amax(stim_resp[i])
        stim_spec_resp.append(np.vstack([stim_spec[i], stim_resp[i]]))
                              
    for j in range(len(stim_spec_resp)):
        spec1_list.append([stim_spec_resp[j][:, n] for n in range(stim_spec_resp[j].shape[-1] - nsegbins - lag)])
        segs_list.append([stim_spec_resp[j][:, (n+lag):(n+nsegbins+lag)] for n in range(stim_spec_resp[j].shape[-1] - nsegbins - lag)])

    for k in range(len(segs_list)):
        indx = [index for index,value in enumerate(segs_list[k]) if (np.mean(value[:nfreqbins, :]) > thresh)]
        #print(type(indx))
        segs_list[k] = [s for s in segs_list[k] if (np.mean(s[:nfreqbins, :]) > thresh)]
        spec1_list[k] = [spec1_list[k][i] for i in indx]

        segs_list[k] = np.stack(segs_list[k], axis=0)
        spec1_list[k] = np.stack(spec1_list[k], axis=0)

    segs = np.vstack(segs_list)
    spec1 = np.vstack(spec1_list)
    
    print('Final shape: {}'.format(segs.shape))
    print('Next shape: {}'.format(spec1.shape))
    return segs, spec1

In [12]:
lag = 16                                                                                                           ####change  

In [13]:
dsegs_16, spec1_16 = prepare_temporal_dset_lag1(stim_spec_16, stim_resp, lag, nsegbins=32, thresh=0.01)

Final shape: (54495, 246, 32)
Next shape: (54495, 246)


In [14]:
spec_next_name = '/mnt/cube/srrudrar/stim_preprocess/temporal_model/B1240/next_list_32_lag_neg16.pkl'              ####change
with open(spec_next_name, 'wb') as f:
    pickle.dump(spec1_16, f)

spec_segs_name = '/mnt/cube/srrudrar/stim_preprocess/temporal_model/B1240/segs_list_32_lag_neg16.pkl'              ####change
with open(spec_segs_name, 'wb') as f:
    pickle.dump(dsegs_16, f)